# Pandas Datareader

# READ THIS FIRST:


### THE STOCK APIS ARE OFTEN CHANGING DUE TO COMPANIES SUCH AS GOOGLE AND YAHOO CHANGING THEIR API ASPECT DUE TO A VARIETY OF FACTORS. CHECK THE PANDAS DATAREADER WEBSITE TO GET INFORMATION ON THE LATEST APIS, SINCE THIS WILL HAVE THE LATEST INFORMATION

## https://pandas-datareader.readthedocs.io/en/latest/

----
** NOTE: Not every geographical location works well with pandas datareader, your firewall may also block it!**

---

Functions from pandas_datareader.data and pandas_datareader.wb extract data from various Internet sources into a pandas DataFrame. Currently the following sources are supported:

* Yahoo! Finance
* Google Finance
* Enigma
* St.Louis FED (FRED)
* Kenneth French’s data library
* World Bank
* OECD
* Eurostat
* Thrift Savings Plan
* Oanda currency historical rate
* Nasdaq Trader symbol definitions (remote_data.nasdaq_symbols)

It should be noted, that various sources support different kinds of data, so not all sources implement the same methods and the data elements returned might also differ.

In [9]:
import pandas_datareader.data as web

import datetime

start = datetime.datetime(2015, 1, 1)

end = datetime.datetime(2017, 1, 1)

# try 'yahoo' if Google doesn't work. make sure to check the website mentioned above
# search QA forums if you have any issues on this, many questions have already been answered there!
# "iex" or "morningstar"
facebook = web.DataReader("FB", 'iex', start, end, api_key='sk_77c0c91beda346c0971dd7e39f7a81f4')

In [10]:
facebook.head()

,open,high,low,close,volume
date,,,,,
2015-08-07,95.38,95.40,93.61,94.30,23199145
2015-08-10,95.68,95.90,93.63,94.15,21624153
2015-08-11,93.73,94.76,92.90,93.62,22491031
2015-08-12,92.70,94.44,91.19,94.19,27120375
2015-08-13,94.05,94.73,93.35,93.43,17451768


### Experimental Options

# NOTE: Google has currently disable this. Check out pandas-datareader online docs for the latest information.

The Options class allows the download of options data from Google Finance.

The get_options_data method downloads options data for specified expiry date and provides a formatted DataFrame with a hierarchical index, so its easy to get to the specific option you want.

Available expiry dates can be accessed from the expiry_dates property.

In [18]:
import pandas as pd
class YahooDailyReader():
    
    def __init__(self, symbol=None, start=None, end=None):
        import datetime, time
        self.symbol = symbol
        
        # initialize start/end dates if not provided
        if end is None:
            end = datetime.datetime.today()
        if start is None:
            start = datetime.datetime(2010,1,1)
        
        self.start = start
        self.end = end
        
        # convert dates to unix time strings
        unix_start = int(time.mktime(self.start.timetuple()))
        day_end = self.end.replace(hour=23, minute=59, second=59)
        unix_end = int(time.mktime(day_end.timetuple()))
        
        url = 'https://finance.yahoo.com/quote/{}/history?'
        url += 'period1={}&period2={}'
        url += '&filter=history'
        url += '&interval=1d'
        url += '&frequency=1d'
        self.url = url.format(self.symbol, unix_start, unix_end)
        
    def read(self):
        import requests, re, json
       
        r = requests.get(self.url)
        
        ptrn = r'root\.App\.main = (.*?);\n}\(this\)\);'
        txt = re.search(ptrn, r.text, re.DOTALL).group(1)
        jsn = json.loads(txt)
        df = pd.DataFrame(
                jsn['context']['dispatcher']['stores']
                ['HistoricalPriceStore']['prices']
                )
        df.insert(0, 'symbol', self.symbol)
        df['date'] = pd.to_datetime(df['date'], unit='s').dt.date
        
        # drop rows that aren't prices
        df = df.dropna(subset=['close'])
        
        df = df[['symbol', 'date', 'high', 'low', 'open', 'close', 
                 'volume', 'adjclose']]
        df = df.set_index('symbol')
        return df
    
ydr = YahooDailyReader('IBM',start=start,end=end)
df = ydr.read()

In [20]:
df.head()

,date,high,low,open,close,volume,adjclose
symbol,,,,,,,
IBM,2016-12-30,166.699997,165.500000,166.440002,165.990005,2952800.0,140.621094
IBM,2016-12-29,166.990005,166.000000,166.020004,166.600006,1663500.0,141.137833
IBM,2016-12-28,167.740005,166.000000,167.289993,166.190002,1757500.0,140.790512
IBM,2016-12-27,167.979996,166.850006,166.979996,167.139999,1397500.0,141.595306
IBM,2016-12-23,167.490005,166.449997,167.000000,166.710007,1701200.0,141.231049


In [19]:
from pandas_datareader.data import Options

fb_options = Options('FB', 'yahoo')

ImmediateDeprecationError: 
Yahoo Options has been immediately deprecated due to large breaks in the API without the
introduction of a stable replacement. Pull Requests to re-enable these data
connectors are welcome.

See https://github.com/pydata/pandas-datareader/issues


In [8]:
data = fb_options.get_options_data(expiry=fb_options.expiry_dates[0])

In [10]:
data.head()

Last     Bid     Ask  Chg  PctChg  \
Strike Expiry     Type Symbol                                                   
5.0    2018-01-19 call FB180119C00005000  149.10  150.00  150.65  0.3    0.20   
                  put  FB180119P00005000    0.01     NaN    0.03  0.0    0.00   
10.0   2018-01-19 call FB180119C00010000  143.20  145.05  145.75 -0.5   -0.35   
                  put  FB180119P00010000    0.01     NaN    0.02  0.0    0.00   
15.0   2018-01-19 call FB180119C00015000  135.80  140.10  140.80  0.0    0.00   

                                           Vol  Open_Int Root  \
Strike Expiry     Type Symbol                                   
5.0    2018-01-19 call FB180119C00005000  50.0    9528.0   FB   
                  put  FB180119P00005000   NaN    3053.0   FB   
10.0   2018-01-19 call FB180119C00010000  40.0    2855.0   FB   
                  put  FB180119P00010000   NaN    2056.0   FB   
15.0   2018-01-19 call FB180119C00015000   NaN     240.0   FB   

                                          Underlying_Price  \
Strike Expiry     Type Symbol                                
5.0    2018-01-19 call FB180119C00005000            155.27   
                  put  FB180119P00005000            155.27   
10.0   2018-01-19 call FB180119C00010000            155.27   
                  put  FB180119P00010000            155.27   
15.0   2018-01-19 call FB180119C00015000            155.27   

                                                         Quote_Time  
Strike Expiry     Type Symbol                                        
5.0    2018-01-19 call FB180119C00005000 2017-07-11 23:35:24.466303  
                  put  FB180119P00005000 2017-07-11 23:35:24.466303  
10.0   2018-01-19 call FB180119C00010000 2017-07-11 23:35:24.466303  
                  put  FB180119P00010000 2017-07-11 23:35:24.466303  
15.0   2018-01-19 call FB180119C00015000 2017-07-11 23:35:24.466303

# FRED

In [21]:
import pandas_datareader.data as web

import datetime

start = datetime.datetime(2010, 1, 1)

end = datetime.datetime(2017, 1, 1)

gdp = web.DataReader("GDP", "fred", start, end)

In [22]:
gdp.head()

,GDP
DATE,
2010-01-01,14721.350
2010-04-01,14926.098
2010-07-01,15079.917
2010-10-01,15240.843
2011-01-01,15285.828
